# Metadata json to netCDF converter
Converting a jason metadata template file to a comtomized netCDF file, which contains only the header without data. 
From the template, users could add the attributes and change the value for each attribute for dimensions, global attributes and variables. 

In [1]:
# Import modules
import os 
import json
from netCDF4 import Dataset

In [2]:
# Define the filename for the input sample json file and output nc/json files
example_json = 'meta_tmpl.json'
output_file = "empty_netcdf"

In [3]:
# A recursive function to change the attribute values.
def change_val(grp_attr):
    for key, value in grp_attr.items():
        if (type(value) is dict):
            #print("\n", "GROUP NAME: " + key)
            x = input("#### Want to change the attributes of '" + key + "'? (Y/Enter): ")
            if x.lower() == 'y':
                # Print the json file
                json_formatted_str = json.dumps(value, indent=2)
                print(json_formatted_str)
                change_val(value)
        else:
            #print ('\t', key, ' = ', value)
            x = input("--> Want to change the '" + key + "' value? (Y/Enter): ")
            if x.lower() == 'y':
                type_str = input("  -->> Enter the data type (int or float): ")
                if type_str ==  "int":
                    grp_attr[key] = int(input("Enter new value for '" + key + "': "))
                elif type_str ==  "float":
                    grp_attr[key] = float(input("Enter new value for '" + key + "': "))
                else:
                    grp_attr[key] = input("Enter new value for '" + key + "': ")
    return

In [4]:
# Open the metadata template json file for read
parentdir = os.getcwd()
jpath = os.path.join(parentdir, 'smpl') 
smpl_json = os.path.join(jpath,example_json)

In [5]:
# Read the json file
with open(smpl_json, 'r') as f:
    mData = json.loads(f.read())

In [6]:
# Print the sample json file
json_formatted_str = json.dumps(mData, indent=2)
print(json_formatted_str)

{
  "dimensions": {
    "time": 1,
    "lat": 9000,
    "lon": 18000
  },
  "variables": {
    "time": {
      "type": "int",
      "dimensions": [
        "time"
      ],
      "attributes": {
        "long_name": "reference time of sst file",
        "units": "seconds since 1981-01-01 00:00:00",
        "comment": "seconds since 1981-01-01 00:00:00",
        "standard_name": "time",
        "calendar": "Gregorian",
        "axis": "T",
        "coverage_content_type": "coordinate"
      }
    },
    "lat": {
      "type": "float",
      "dimensions": [
        "lat"
      ],
      "attributes": {
        "long_name": "latitude",
        "comment": "Latitude for locating data",
        "units": "degrees_north",
        "axis": "Y",
        "valid_min": -90,
        "valid_max": 90,
        "standard_name": "latitude",
        "coverage_content_type": "coordinate"
      }
    },
    "lon": {
      "type": "float",
      "dimensions": [
        "lon"
      ],
      "attributes": {
     

# Review and change "dimensions"

In [10]:
# dimensions fields
dims = mData['dimensions']
dims

{'time': 1, 'lat': 90, 'lon': 180}

In [8]:
# Changing the key of dimensions
# dims['ni'] = dims.pop('lat')
# dims['nj'] = dims.pop('lon')

In [9]:
# Loop through the dimensions, change the attribute value as needed.
change_val(dims)

--> Want to change the 'time' value? (Y/Enter): 
--> Want to change the 'lat' value? (Y/Enter): y
  -->> Enter the data type (int or float): int
Enter new value for 'lat': 90
--> Want to change the 'lon' value? (Y/Enter): y
  -->> Enter the data type (int or float): int
Enter new value for 'lon': 180


# Review and change "global attributes"

In [11]:
# Global attribute dict
glb_attr = mData['global_attributes']
glb_attr

{'title': 'VIIRS L2P Sea Surface Skin Temperature',
 'summary': 'Sea surface temperature (SST) retrievals produced at the NASA',
 'Conventions': 'CF-1.7, ACDD-1.3',
 'standard_name_vocabulary': 'NetCDF Climate and Forecast (CF) Metadata Convention'}

In [12]:
# Loop through the global attributes, change the attribute value as needed.
change_val(glb_attr)

--> Want to change the 'title' value? (Y/Enter): y
  -->> Enter the data type (int or float): 
Enter new value for 'title': sst
--> Want to change the 'summary' value? (Y/Enter): 
--> Want to change the 'Conventions' value? (Y/Enter): 
--> Want to change the 'standard_name_vocabulary' value? (Y/Enter): 


In [13]:
# Add new global attributes
isAdd_ga = input("-- Want to Add a new global attribute (Y/Enter): ")
while isAdd_ga.lower() == 'y':
    newKey = input("Enter the new key string: ")
    newAttr = input("Enter the new attribute value: ")
    glb_attr[newKey] = newAttr
    isAdd_ga = input("-- Want to Add another new global attribute (Y/Enter): ")
    if isAdd_ga.lower() != 'y': break

-- Want to Add a new global attribute (Y/Enter): y
Enter the new key string: id
Enter the new attribute value: test1
-- Want to Add another new global attribute (Y/Enter): y
Enter the new key string: platform
Enter the new attribute value: modis
-- Want to Add another new global attribute (Y/Enter): 


In [14]:
json_formatted_str = json.dumps(glb_attr, indent=2)
print(json_formatted_str)

{
  "title": "sst",
  "summary": "Sea surface temperature (SST) retrievals produced at the NASA",
  "Conventions": "CF-1.7, ACDD-1.3",
  "standard_name_vocabulary": "NetCDF Climate and Forecast (CF) Metadata Convention",
  "id": "test1",
  "platform": "modis"
}


# Review and change "variables"

In [15]:
# Variables group
var_attr = mData['variables']
#var_attr
print([key for key in var_attr.keys()])

['time', 'lat', 'lon', 'example_var', 'crs']


In [16]:
# Add the new variable to variables attribute
isAdd_var = input("-- Want to Add a new variable (Y/Enter): ")
while isAdd_var.lower() == 'y':
    newVar = input("Enter the new variable name: ")
#    newVal = input("Enter the new variable struct: ")
    var_attr[newVar]= var_attr["example_var"]
    isAdd_var = input("-- Want to Add another variable (Y/Enter): ")
    if isAdd_var.lower() != 'y': break

-- Want to Add a new variable (Y/Enter): y
Enter the new variable name: sst
-- Want to Add another variable (Y/Enter): y
Enter the new variable name: l2p_flags
-- Want to Add another variable (Y/Enter): 


In [17]:
# Remove the example_var variable
if 'example_var' in var_attr.keys(): 
    del var_attr['example_var']
    print([key for key in var_attr.keys()])

['time', 'lat', 'lon', 'crs', 'sst', 'l2p_flags']


In [18]:
# Loop through the variables
change_val(var_attr)

#### Want to change the attributes of 'time'? (Y/Enter): 
#### Want to change the attributes of 'lat'? (Y/Enter): 
#### Want to change the attributes of 'lon'? (Y/Enter): 
#### Want to change the attributes of 'crs'? (Y/Enter): 
#### Want to change the attributes of 'sst'? (Y/Enter): 
#### Want to change the attributes of 'l2p_flags'? (Y/Enter): 


In [19]:
# reviewing all the variables
# var_attr
json_formatted_str = json.dumps(var_attr, indent=2)
print(json_formatted_str)

{
  "time": {
    "type": "int",
    "dimensions": [
      "time"
    ],
    "attributes": {
      "long_name": "reference time of sst file",
      "units": "seconds since 1981-01-01 00:00:00",
      "comment": "seconds since 1981-01-01 00:00:00",
      "standard_name": "time",
      "calendar": "Gregorian",
      "axis": "T",
      "coverage_content_type": "coordinate"
    }
  },
  "lat": {
    "type": "float",
    "dimensions": [
      "lat"
    ],
    "attributes": {
      "long_name": "latitude",
      "comment": "Latitude for locating data",
      "units": "degrees_north",
      "axis": "Y",
      "valid_min": -90,
      "valid_max": 90,
      "standard_name": "latitude",
      "coverage_content_type": "coordinate"
    }
  },
  "lon": {
    "type": "float",
    "dimensions": [
      "lon"
    ],
    "attributes": {
      "long_name": "longitude",
      "comment": "Longitude for locating data",
      "units": "degrees_east",
      "axis": "X",
      "valid_min": -180,
      "valid_

# Write the new metadata to netCDF and json files to out_dir directory

In [20]:
# Define the output netCDF file path
oPath = os.path.join(parentdir, 'out_dir') 
empty_nc = os.path.join(oPath, output_file + '.nc')
empty_nc

'C:\\Users\\wenhaoli\\Documents\\GitHub\\github_whl\\out_dir\\empty_netcdf.nc'

In [21]:
if not os.path.exists(oPath):
    os.makedirs(oPath)

In [22]:
# Create an empty netCDF file
with Dataset(empty_nc, "w") as nc:
    # Add dimensions
    for dim_name, dim_size in mData["dimensions"].items():
        nc.createDimension(dim_name, dim_size)

    # Add variables and their attributes
    for var_name, var_attrs in mData["variables"].items():
        var = nc.createVariable(var_name, var_attrs["type"], var_attrs["dimensions"])
        for attr_name, attr_value in var_attrs["attributes"].items():
            setattr(var, attr_name, attr_value)

    # Add global attributes
    for global_attr_name, global_attr_value in mData["global_attributes"].items():
        setattr(nc, global_attr_name, global_attr_value)

print(f"Empty netCDF file '{empty_nc}' created successfully with embedded JSON metadata attributes.")

Empty netCDF file 'C:\Users\wenhaoli\Documents\GitHub\github_whl\out_dir\empty_netcdf.nc' created successfully with embedded JSON metadata attributes.


In [23]:
# system call to display the customized meta info from the netCDF. 
!ncdump -h out_dir\empty_netcdf.nc

netcdf out_dir\\empty_netcdf {
dimensions:
	time = 1 ;
	lat = 90 ;
	lon = 180 ;
variables:
	int time(time) ;
		time:long_name = "reference time of sst file" ;
		time:units = "seconds since 1981-01-01 00:00:00" ;
		time:comment = "seconds since 1981-01-01 00:00:00" ;
		time:standard_name = "time" ;
		time:calendar = "Gregorian" ;
		time:axis = "T" ;
		time:coverage_content_type = "coordinate" ;
	double lat(lat) ;
		lat:long_name = "latitude" ;
		lat:comment = "Latitude for locating data" ;
		lat:units = "degrees_north" ;
		lat:axis = "Y" ;
		lat:valid_min = -90. ;
		lat:valid_max = 90. ;
		lat:standard_name = "latitude" ;
		lat:coverage_content_type = "coordinate" ;
	double lon(lon) ;
		lon:long_name = "longitude" ;
		lon:comment = "Longitude for locating data" ;
		lon:units = "degrees_east" ;
		lon:axis = "X" ;
		lon:valid_min = -180. ;
		lon:valid_max = 180. ;
		lon:standard_name = "longitude" ;
		lon:coverage_content_type = "coordinate" ;
	int crs(lat, lon) ;
		crs:grid_mapping_name 

In [24]:
# Output the modified metadata dict to a json file.
out_json = os.path.join(oPath, 'meta_tmpl' + '.json')
with open(out_json, 'w', encoding='utf-8') as f:
    json.dump(mData, f, ensure_ascii=False, indent=4)